# Import necessary dependencies and settings

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

#pd.options.display.max_colwidth = 200
#import matplotlib inline

# Load the data

In [3]:

data=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/data2.csv',encoding= 'unicode_escape')
df=pd.DataFrame(data)
df=df.sample(frac=1)
corpus_df=df.Question
labels=df.Label
upsc=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/upsc.csv',encoding= 'unicode_escape')

# Simple text pre-processing

In [ ]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [ ]:
norm_corpus = normalize_corpus(corpus_df)
#upsc=normalize_corpus(upsc)


# Bag of Words Model

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

In [ ]:
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

# Bag of N-Grams Model

In [ ]:
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

# TF-IDF Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

# Document Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

# Apply SVM 

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle

#X = df['cleaned']
Y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(similarity_df, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
from sklearn.svm import LinearSVC
def support_vector_machine(X_train, y, X_predict):
    lin_clf = LinearSVC()
    lin_clf.fit(X_train, y)
    prediction = lin_clf.predict(X_predict)
    return prediction

y_pred=support_vector_machine(X_train,y_train,X_test)
#xtest=pca.transform(x_test)
from sklearn.metrics import classification_report,accuracy_score
#y_pred=cls.predict(vectorization.transform(xtest))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

## Clustering documents using similarity features

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2', 
                         'Distance', 'Cluster Size'], dtype='object')

In [ ]:
plt.figure(figsize=(8, 3))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data point')
plt.ylabel('Distance')
dendrogram(Z)
plt.axhline(y=1.0, c='k', ls='--', lw=0.5)

In [ ]:
from scipy.cluster.hierarchy import fcluster
max_dist = 1.0

cluster_labels = fcluster(Z, max_dist, criterion='distance')
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([df, cluster_labels], axis=1)

# Topic Models

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
pd.options.display.max_colwidth = 200

%matplotlib inline

lda = LatentDirichletAllocation(n_components=3, max_iter=10000, random_state=0)
dt_matrix = lda.fit_transform(cv_matrix)
features = pd.DataFrame(dt_matrix, columns=['T1', 'T2', 'T3'])
features

## Show topics and their weights

In [ ]:
tt_matrix = lda.components_
for topic_weights in tt_matrix:
    topic = [(token, weight) for token, weight in zip(vocab, topic_weights)]
    topic = sorted(topic, key=lambda x: -x[1])
    topic = [item for item in topic if item[1] > 0.6]
    print(topic)
    print()


## Clustering documents using topic model features

In [ ]:
pd.concat([df, cluster_labels], axis=1)

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle

#X = df['cleaned']
Y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(similarity_df, Y, test_size=0.25)
# instead of doing these steps one at a time, we can use a pipeline to complete them all at once
from sklearn.svm import LinearSVC
def support_vector_machine(X_train, y, X_predict):
    lin_clf = LinearSVC()
    lin_clf.fit(X_train, y)
    prediction = lin_clf.predict(X_predict)
    return prediction

y_pred=support_vector_machine(X_train,y_train,X_test)
#xtest=pca.transform(x_test)
from sklearn.metrics import classification_report,accuracy_score
#y_pred=cls.predict(vectorization.transform(xtest))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))